<a href="https://colab.research.google.com/github/Gokul0Krishna/pytorchpractice/blob/master/attempt_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip "/content/synthetic.zip"
!unzip "/content/washer.v1i.multiclass.zip"
!unzip "/content/test.zip"
!unzip "/content/washer.v4i.multiclass.zip"

unzip:  cannot find or open /content/synthetic.zip, /content/synthetic.zip.zip or /content/synthetic.zip.ZIP.
unzip:  cannot find or open /content/washer.v1i.multiclass.zip, /content/washer.v1i.multiclass.zip.zip or /content/washer.v1i.multiclass.zip.ZIP.
unzip:  cannot find or open /content/test.zip, /content/test.zip.zip or /content/test.zip.ZIP.
unzip:  cannot find or open /content/washer.v4i.multiclass.zip, /content/washer.v4i.multiclass.zip.zip or /content/washer.v4i.multiclass.zip.ZIP.


In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torchvision import models,transforms
from torch.utils.data import DataLoader,Dataset

KeyboardInterrupt: 

In [ ]:
l=['washer','nut']
realimages=[]
reallabels=[]
for i in os.listdir("/content/washer.v4i.multiclass"):
  for j in os.listdir("/content/washer.v4i.multiclass/"+i):
    if j.endswith(".jpg") == True:
      realimages.append("/content/washer.v4i.multiclass/"+i+"/"+j)
      reallabels.append("washer")

for i in os.listdir("/content/washer.v1i.multiclass"):
    if i.endswith(".txt") == False:
      for j in os.listdir("/content/washer.v1i.multiclass/"+i):
        if j.endswith(".jpg") == True:
          realimages.append("/content/washer.v1i.multiclass/"+i+"/"+j)
          reallabels.append("washer")
a=0
for i in os.listdir("/content/test"):
  for j in os.listdir("/content/test/"+i):
      if a>250:
          break
      realimages.append("/content/test/"+i+"/"+j)
      reallabels.append("nut")
      a+=1

In [ ]:
syntheticimages=[]
syntheticlabels=[]
for i in os.listdir("/content/synthetic"):
  for j in os.listdir("/content/synthetic/"+i):
      syntheticimages.append("/content/synthetic/"+i+"/"+j)
      syntheticlabels.append(i)

In [ ]:
dfs=pd.DataFrame({'X':syntheticimages,'Y':syntheticlabels})
dfs=pd.get_dummies(dfs,columns=['Y'],dtype=float)
dfs.columns = dfs.columns.str.replace("Y_", "")

dfr=pd.DataFrame({'X':realimages,'Y':reallabels})
dfr=pd.get_dummies(dfr,columns=['Y'],dtype=float)
dfr.columns = dfr.columns.str.replace("Y_", "")

In [ ]:
class customdataset(Dataset):
    def __init__(self,x,y,transform=None):
        self.x=x
        self.y=y
        self.transform=transform

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        imgpath=self.x.iloc[index]
        if self.transform:
            image=Image.open(imgpath).convert("RGB")
            image=self.transform(image)
        label_row = self.y.iloc[index, 0:2]
        label_tensor = torch.tensor(label_row.values.astype(np.float32))
        label_index = torch.argmax(label_tensor).item()

        return image,label_index

In [ ]:
synthetic_train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomRotation(30),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
testmodel = models.densenet201(pretrained=False)

In [ ]:
from sklearn.model_selection import train_test_split
xstrain, xstest,ystrain,ystest = train_test_split(dfs['X'],dfs[l],random_state=42, test_size=0.2)
traindataset=customdataset(x=xstrain,y=ystrain,transform=synthetic_train_transforms)
valdataset=customdataset(x=xstest,y=ystest,transform=synthetic_train_transforms)

trainloader=DataLoader(traindataset,batch_size=32,shuffle=True)
valloader=DataLoader(valdataset,batch_size=32,shuffle=True)

In [ ]:
testmodel.classifier = torch.nn.Linear(testmodel.classifier.in_features, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(testmodel.parameters(), lr=0.0001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
testmodel = testmodel.to(device)

In [ ]:
num_epochs = 3

for epoch in range(num_epochs):
    testmodel.train()
    trunningloss,vrunningloss=0.0,0.0
    ttotal,vtotal = 0,0
    tacc,vacc=0.0,0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = testmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        trunningloss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        ttotal += labels.size(0)
        tacc += (predicted == labels).sum().item()


    testmodel.eval()
    with torch.no_grad():
      for inputs, labels in valloader:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = testmodel(inputs)
          loss = criterion(outputs, labels)
          vrunningloss += loss.item()

          _, predicted = torch.max(outputs.data, 1)  # Get predicted class
          vtotal += labels.size(0)
          vacc += (predicted == labels).sum().item()

    print("="*20)
    print(f"""Epoch {epoch+1}
          train
          Loss: {trunningloss/len(trainloader)}
          Accuracy: {tacc*100/ttotal}
          val
          Loss: {vrunningloss/len(valloader)}
          Accuracy: {vacc*100/vtotal}
          """)

In [ ]:
xstrain, xstest,ystrain,ystest = train_test_split(dfs['X'],dfs[l],random_state=42, test_size=0.3)
straindataset=customdataset(x=xstrain,y=ystrain,transform=synthetic_train_transforms)
svaldataset=customdataset(x=xstest,y=ystest,transform=synthetic_train_transforms)

xrtrain, xrtest,yrtrain,yrtest = train_test_split(dfr['X'],dfr[l],random_state=42, test_size=0.3)
rtraindataset=customdataset(x=xstrain,y=ystrain,transform=synthetic_train_transforms)
rvaldataset=customdataset(x=xstest,y=ystest,transform=synthetic_train_transforms)


In [ ]:
mixedtraindataset = torch.utils.data.ConcatDataset([straindataset, rtraindataset])
# mixedtrainloader = DataLoader(mixedtraindataset, batch_size=64, shuffle=True)

mixedvaldataset = torch.utils.data.ConcatDataset([svaldataset, rvaldataset])
# mixedvalloader = DataLoader(mixedvaldataset, batch_size=64, shuffle=True)


In [ ]:
real_weight = 3.0  # Higher weight for real data
synthetic_weight = 1.0
weights = torch.tensor([synthetic_weight] * len(straindataset) + [real_weight] * len(rtraindataset))
sampler = torch.utils.data.WeightedRandomSampler(weights, len(weights))
tweighted_mixed_loader = DataLoader(mixedtraindataset, batch_size=32, sampler=sampler)

weights = torch.tensor([synthetic_weight] * len(svaldataset) + [real_weight] * len(rvaldataset))
sampler = torch.utils.data.WeightedRandomSampler(weights, len(weights))
vweighted_mixed_loader = DataLoader(mixedvaldataset, batch_size=32, sampler=sampler)

In [ ]:
for param in testmodel.features.parameters():
    param.requires_grad = False
class_weights = torch.tensor([1.0, 3.0])
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = torch.optim.Adam(testmodel.parameters(), lr=0.0001)

In [ ]:
num_epochs = 4

for epoch in range(num_epochs):
    testmodel.train()
    trunningloss,vrunningloss=0.0,0.0
    ttotal,vtotal = 0,0
    tacc,vacc=0.0,0.0
    for inputs, labels in tweighted_mixed_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = testmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        trunningloss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        ttotal += labels.size(0)
        tacc += (predicted == labels).sum().item()


    testmodel.eval()
    with torch.no_grad():
      for inputs, labels in vweighted_mixed_loader:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = testmodel(inputs)
          loss = criterion(outputs, labels)
          vrunningloss += loss.item()

          _, predicted = torch.max(outputs.data, 1)  # Get predicted class
          vtotal += labels.size(0)
          vacc += (predicted == labels).sum().item()

    print("="*20)
    print(f"""Epoch {epoch+1}
          train
          Loss: {trunningloss/len(tweighted_mixed_loader)}
          Accuracy: {tacc*100/ttotal}
          val
          Loss: {vrunningloss/len(vweighted_mixed_loader)}
          Accuracy: {vacc*100/vtotal}
          """)

In [ ]:
real_train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [ ]:
rtraindataset=customdataset(x=xstrain,y=ystrain,transform=real_train_transforms)
rvaldataset=customdataset(x=xstest,y=ystest,transform=real_train_transforms)
mixedtrainloader = DataLoader(mixedtraindataset, batch_size=16, shuffle=True)
mixedvalloader = DataLoader(mixedvaldataset, batch_size=16, shuffle=True)

In [ ]:
for param in testmodel.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(testmodel.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 4

for epoch in range(num_epochs):
    testmodel.train()
    trunningloss,vrunningloss=0.0,0.0
    ttotal,vtotal = 0,0
    tacc,vacc=0.0,0.0
    for inputs, labels in mixedtrainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = testmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        trunningloss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        ttotal += labels.size(0)
        tacc += (predicted == labels).sum().item()


    testmodel.eval()
    with torch.no_grad():
      for inputs, labels in mixedvalloader:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = testmodel(inputs)
          loss = criterion(outputs, labels)
          vrunningloss += loss.item()

          _, predicted = torch.max(outputs.data, 1)  # Get predicted class
          vtotal += labels.size(0)
          vacc += (predicted == labels).sum().item()

    print("="*20)
    print(f"""Epoch {epoch+1}
          train
          Loss: {trunningloss/len(tweighted_mixed_loader)}
          Accuracy: {tacc*100/ttotal}
          val
          Loss: {vrunningloss/len(vweighted_mixed_loader)}
          Accuracy: {vacc*100/vtotal}
          """)